In [1]:
#!/usr/bin/python -W ignore::DeprecationWarning

import sys
import pickle
sys.path.append("../tools/")
from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data
import pandas as pd
import sys
import pickle
import csv
import matplotlib.pyplot as plt

sys.path.append("../tools/")
from feature_format import featureFormat, targetFeatureSplit
#from poi_data import *
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit

from numpy import mean

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

from sklearn.metrics import accuracy_score, precision_score, recall_score


In [2]:
#% ## Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".
#features_list = ['poi','salary'] # You will need to use more features

############# Task 1: Select what features you'll use.############


In [2]:
target_label = 'poi'

email_features_list = [
    'from_messages',
    'from_poi_to_this_person',
    'from_this_person_to_poi',
    'shared_receipt_with_poi',
    'to_messages',
    ]
    
financial_features_list = [
    'bonus',
    'deferral_payments',
    'deferred_income',
    'director_fees',
    'exercised_stock_options',
    'expenses',
    'loan_advances',
    'long_term_incentive',
    'other',
    'restricted_stock',
    'restricted_stock_deferred',
    'salary',
    'total_payments',
    'total_stock_value',
]

features_list = [target_label] + financial_features_list + email_features_list

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "rb") as data_file:
    data_dict = pickle.load(data_file)

In [3]:
import numpy as np
df = pd.read_pickle('final_project_dataset.pkl')
df=pd.DataFrame.from_dict(df)
# df.pivot()
# df.melt(id_vars=features_list[1:])
df=df.T
df = pd.DataFrame(df)
# print(df.head(1))
print(type(df))
df.replace(to_replace='NaN', value=np.nan, inplace=True)


# df=df.replace(to_replace=np.nan, value=0)
dataf=df.fillna(0)
# df=df.to_dict
outliers = ['TOTAL', 'THE TRAVEL AGENCY IN THE PARK', 'LOCKHART EUGENE E']
for i in outliers:
    dataf=dataf.drop(i,axis=0)
dataf

<class 'pandas.core.frame.DataFrame'>


,salary,to_messages,deferral_payments,total_payments,loan_advances,bonus,email_address,restricted_stock_deferred,deferred_income,total_stock_value,...,from_poi_to_this_person,exercised_stock_options,from_messages,other,from_this_person_to_poi,poi,long_term_incentive,shared_receipt_with_poi,restricted_stock,director_fees
METTS MARK,365788.0,807.0,0.0,1061827.0,0.0,600000.0,mark.metts@enron.com,0.0,0.0,585062.0,...,38.0,0.0,29.0,1740.0,1.0,False,0.0,702.0,585062.0,0.0
BAXTER JOHN C,267102.0,0.0,1295738.0,5634343.0,0.0,1200000.0,0,0.0,-1386055.0,10623258.0,...,0.0,6680544.0,0.0,2660303.0,0.0,False,1586055.0,0.0,3942714.0,0.0
ELLIOTT STEVEN,170941.0,0.0,0.0,211725.0,0.0,350000.0,steven.elliott@enron.com,0.0,-400729.0,6678735.0,...,0.0,4890344.0,0.0,12961.0,0.0,False,0.0,0.0,1788391.0,0.0
CORDES WILLIAM R,0.0,764.0,0.0,0.0,0.0,0.0,bill.cordes@enron.com,0.0,0.0,1038185.0,...,10.0,651850.0,12.0,0.0,0.0,False,0.0,58.0,386335.0,0.0
HANNON KEVIN P,243293.0,1045.0,0.0,288682.0,0.0,1500000.0,kevin.hannon@enron.com,0.0,-3117011.0,6391065.0,...,32.0,5538001.0,32.0,11350.0,21.0,True,1617011.0,1035.0,853064.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GRAMM WENDY L,0.0,0.0,0.0,119292.0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,False,0.0,0.0,0.0,119292.0
CAUSEY RICHARD A,415189.0,1892.0,0.0,1868758.0,0.0,1000000.0,richard.causey@enron.com,0.0,-235000.0,2502063.0,...,58.0,0.0,49.0,307895.0,12.0,True,350000.0,1585.0,2502063.0,0.0
TAYLOR MITCHELL S,265214.0,533.0,227449.0,1092663.0,0.0,600000.0,mitchell.taylor@enron.com,0.0,0.0,3745048.0,...,0.0,3181250.0,29.0,0.0,0.0,False,0.0,300.0,563798.0,0.0
DONAHUE JR JEFFREY M,278601.0,865.0,0.0,875760.0,0.0,800000.0,jeff.donahue@enron.com,0.0,-300000.0,1080988.0,...,188.0,765920.0,22.0,891.0,11.0,False,0.0,772.0,315068.0,0.0
